In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import numpy as np
import re

In [2]:
txt_file = '/Users/elisanobile/Documents/IIASA/disruptsc_v1/output/interacting-hazards_v2.txt'
results_folder = '/Users/elisanobile/Documents/IIASA/disruptsc_v1/output/Cambodia/'

In [3]:
# Read the text file
with open(txt_file, 'r') as file:
    lines = file.readlines()

lines

['# BASELINE\n',
 '\n',
 'events:\n',
 '  - type: transport_disruption\n',
 '    description_type: edge_attributes\n',
 '    attribute: flood_disruption\n',
 '    values: ["Saigon"]\n',
 '    start_time: 1\n',
 '    duration: 1\n',
 '  - type: transport_disruption\n',
 '    description_type: edge_attributes\n',
 '    attribute: flood_disruption\n',
 '    values: ["Siem Bok (Mekong)"]\n',
 '    start_time: 1\n',
 '    duration: 1\n',
 '  - type: transport_disruption\n',
 '    description_type: edge_attributes\n',
 '    attribute: flood_disruption\n',
 '    values: ["St. Sen (Tonlè Sap)"]\n',
 '    start_time: 1\n',
 '    duration: 1\n',
 '\n',
 'Exporting loss time series of households per region sector to /Users/elisanobile/Documents/IIASA/disruptsc_v1/output/Cambodia/20250616_110722\n',
 'Exporting loss time series of countries to /Users/elisanobile/Documents/IIASA/disruptsc_v1/output/Cambodia/20250616_110722\n',
 'Cumulated household loss: 29.67 mUSD\n',
 'Cumulated country loss: 5.0

In [70]:
scenarios = [
    {"name": "Simultaneous", "folder": "20250616_110722"}, # 3 basins
    {"name": "Saigon only", "folder": "20250616_100610"}, # 1 basin
    {"name": "Mekong only", "folder": "20250616_102153"}, # 1 basin
    {"name": "Tonlè Sap only", "folder": "20250616_105406"}, # 1 basin
    {"name": "Cascading", "folder": "20250616_124445"}, # Saigon first + Mekong second + Tonlè Sap third (3 basins)
    {"name": "Mixed", "folder": "20250616_114510"}, # Saigon first then (Mekong + Tonlè Sap) (3 basins)
]

In [71]:
summary_data = []

for scenario in scenarios:
    folder_path = os.path.join(results_folder, scenario["folder"])
    summary_file = os.path.join(folder_path, "loss_summary.csv")
    
    if os.path.exists(summary_file):
        df = pd.read_csv(summary_file)
        summary_data.append({
            "scenario": scenario["name"],
            "folder": scenario["folder"],
            "household_loss": df["households"].iloc[0],
            "country_loss": df["countries"].iloc[0],
            "total_loss": df["households"].iloc[0] + df["countries"].iloc[0]
        })

# Create summary DataFrame
summary_df = pd.DataFrame(summary_data)
print(f"\nLoaded {len(summary_df)} scenarios")
summary_df


Loaded 6 scenarios


,scenario,folder,household_loss,country_loss,total_loss
0,Simultaneous,20250616_110722,29.670817,5.006258,34.677076
1,Saigon only,20250616_100610,4.812271,0.873681,5.685951
2,Mekong only,20250616_102153,22.467927,7.802255,30.270182
3,Tonlè Sap only,20250616_105406,3.645360,0.786769,4.432129
4,Cascading,20250616_124445,31.247358,10.319109,41.566467
5,Mixed,20250616_114510,29.504714,6.828499,36.333213


In [72]:
time_series_data = []

for scenario in scenarios:
    folder_path = os.path.join(results_folder, scenario["folder"])
    ts_file = os.path.join(folder_path, "loss_per_country.csv")
    
    if os.path.exists(ts_file):
        df = pd.read_csv(ts_file)
        df["scenario"] = scenario["name"]
        df["folder"] = scenario["folder"]
        time_series_data.append(df)


ts_df = pd.concat(time_series_data, ignore_index=True)
ts_df

,time_step,country,loss,scenario,folder
0,0,AFR,0.000000,Simultaneous,20250616_110722
1,0,AME,0.000000,Simultaneous,20250616_110722
2,0,ASI,0.000000,Simultaneous,20250616_110722
3,0,EUR,0.000000,Simultaneous,20250616_110722
4,0,OCE,0.000000,Simultaneous,20250616_110722
...,...,...,...,...,...
380,10,ASI,0.000446,Mixed,20250616_114510
381,10,EUR,0.001196,Mixed,20250616_114510
382,10,OCE,0.000030,Mixed,20250616_114510
383,10,THA,0.000011,Mixed,20250616_114510


In [86]:
fig1 = go.Figure()

fig1.add_trace(go.Bar(
    name='Household loss',
    x=summary_df['scenario'],
    y=summary_df['household_loss'],
    marker_color='lightcoral',
    text=summary_df['household_loss'].round(1),
    textposition='inside'
))

fig1.add_trace(go.Bar(
    name='Country loss',
    x=summary_df['scenario'], 
    y=summary_df['country_loss'],
    marker_color='steelblue',
    text=summary_df['country_loss'].round(1),
    textposition='inside'
))

fig1.update_layout(
    xaxis_title='Scenario',
    yaxis_title='Cumulative loss (million USD)',
    barmode='stack',
    height=500,
    xaxis_tickangle=-45,
    showlegend=True,
    font=dict(size=16)
)

fig1.show()

In [74]:
if 'ts_df' in locals():
    total_by_time = ts_df.groupby(['scenario', 'time_step'])['loss'].sum().reset_index()
    
    fig2 = px.line(
        total_by_time,
        x='time_step',
        y='loss', 
        color='scenario',
        labels={'time_step': 'Weeks', 'loss': 'Total loss (million USD)'},
        markers=True,
        height=500
    )
    
    fig2.update_layout(
        legend=dict(orientation="v", yanchor="top", y=1, xanchor="left", x=1.02)
    )
    
    fig2.show()

In [75]:
if 'ts_df' in locals():
    country_totals = ts_df.groupby(['scenario', 'country'])['loss'].sum().reset_index()
    
    fig3 = px.bar(
        country_totals,
        x='country',
        y='loss',
        labels={'country': 'Countries', 'loss': 'Trade loss (million USD)'},
        color='scenario', 
        barmode='group',
        height=400,
    )   
    
    fig3.update_layout(
        xaxis_tickangle=-45,
        height=400,
        width=1000,
        font=dict(size=18))
    
    fig3.update_layout(
    legend=dict(
        title="",
        orientation="v", 
        yanchor="top", 
        y=1, 
        xanchor="left", 
        x=1.02
    )
)
    fig3.show()

In [76]:
if 'ts_df' in locals():
    fig4 = px.line(
        ts_df,
        x='time_step',
        y='loss',
        color='country',
        facet_col='scenario',
        facet_col_wrap=2,
        height=800,
        markers=True,
        labels={'time_step': 'Weeks', 'loss': 'Country loss (million USD)'}
    )
    
    fig4.update_layout(font=dict(size=10))
    fig4.show()

In [77]:
ranked = summary_df.sort_values('total_loss', ascending=False)

print(f"Total scenarios: {len(ranked)}")
for i, row in ranked.iterrows():
    print(f"{row.name+1:2d}. {row['scenario']:30s} {row['total_loss']:6.1f} mUSD")

Total scenarios: 6
 5. Cascading                        41.6 mUSD
 6. Mixed                            36.3 mUSD
 1. Simultaneous                     34.7 mUSD
 3. Mekong only                      30.3 mUSD
 2. Saigon only                       5.7 mUSD
 4. Tonlè Sap only                    4.4 mUSD


In [78]:
sector_info_dict = {
    'CRO': {'name': 'Crops', 'type': 'agriculture'},
    'LIV': {'name': 'Livestock and Poultry', 'type': 'agriculture'},
    'FOR': {'name': 'Forestry and logging', 'type': 'primary'},
    'FIS': {'name': 'Fishing and aquaculture', 'type': 'primary'},
    'MIN': {'name': 'Mining and quarrying', 'type': 'primary'},
    'MFO': {'name': 'Manufacture of food products', 'type': 'manufacturing'},
    'MBE': {'name': 'Manufacture of beverages', 'type': 'manufacturing'},
    'MTO': {'name': 'Manufacture of tobacco products', 'type': 'manufacturing'},
    'MTE': {'name': 'Manufacture of textiles, leather, wearing apparel and footwear', 'type': 'manufacturing'},
    'MWO': {'name': 'Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials', 'type': 'manufacturing'},
    'MPA': {'name': 'Manufacture of paper and paper products; Printing and reproduction of recorded media', 'type': 'manufacturing'},
    'MCO': {'name': 'Manufacture of coke and refined petroleum products', 'type': 'manufacturing'},
    'MCH': {'name': 'Manufacture of chemicals and chemical products', 'type': 'manufacturing'},
    'MPH': {'name': 'Manufacture of pharmaceuticals, medicinal chemical and botanical products', 'type': 'manufacturing'},
    'MRU': {'name': 'Manufacture of rubber and plastic products', 'type': 'manufacturing'},
    'MNM': {'name': 'Manufacture of other non-metallic mineral products', 'type': 'manufacturing'},
    'MBA': {'name': 'Manufacture of basic metals', 'type': 'manufacturing'},
    'MFA': {'name': 'Manufacture of fabricated metal products, except machinery and equipment', 'type': 'manufacturing'},
    'MEL': {'name': 'Manufacture of electrical equipment, of computer, electronic and optical products', 'type': 'manufacturing'},
    'MMA': {'name': 'Manufacture of machinery and equipment n.e.c.; Repair and installation of machinery and equipement; Manufacture of motor vehicles, trailers and semi-trailers', 'type': 'manufacturing'},
    'MTR': {'name': 'Manufacture of other transport equipment', 'type': 'manufacturing'},
    'MFU': {'name': 'Manufacture of furniture', 'type': 'manufacturing'},
    'MAO': {'name': 'Other manufacturing', 'type': 'manufacturing'},
    'ELE': {'name': 'Electricity, gas, steam and air conditioning supply; Water collection, treatment and supply; Sewerage', 'type': 'utility'},
    'WAS': {'name': 'Waste collection, treatment and disposal activities; materials recovery; Remediation activities and other waste management services', 'type': 'utility'},
    'CON': {'name': 'Construction of buildings; Civil engineering; Specialized construction activities', 'type': 'construction'},
    'TRW': {'name': 'Wholesale trade, except of motor vehicles and motorcycles', 'type': 'trade'},
    'TRR': {'name': 'Retail trade, except of motor vehicles and motorcycles', 'type': 'trade'},
    'TRL': {'name': 'Land transport and transport via pipelines; Water transport; Air transport; Warehousing and support activities for transportation', 'type': 'transport'},
    'POS': {'name': 'Postal and courier activities', 'type': 'services'},
    'ACF': {'name': 'Accommodation; Food and beverage service activities', 'type': 'services'},
    'PUB': {'name': 'Publishing activities', 'type': 'services'},
    'VID': {'name': 'Motion picture, video and television program production, sound recording and  music publishing activities', 'type': 'services'},
    'TEL': {'name': 'Programming and broadcasting activities; Telecommunications; Computer programming, consultancy and related activities', 'type': 'services'},
    'INF': {'name': 'Information service activities', 'type': 'services'},
    'FIN': {'name': 'Financial service activities, except insurance and pension funding', 'type': 'services'},
    'INS': {'name': 'Insurance, reinsurance and pension funding, except compulsory social security; Activities auxiliary to financial service and insurance activities', 'type': 'services'},
    'REA': {'name': 'Real estate activities', 'type': 'services'},
    'ACC': {'name': 'Legal and accounting activities', 'type': 'services'},
    'CST': {'name': 'Activities of head offices; management consultancy activities', 'type': 'services'},
    'ARC': {'name': 'Architectural and engineering activities; technical testing and analysis; Scientific research and development; Advertising and market research; Other professional, scientific and technical activities', 'type': 'services'},
    'VET': {'name': 'Veterinary activities', 'type': 'services'},
    'REN': {'name': 'Rental and leasing activities', 'type': 'services'},
    'EMP': {'name': 'Employment activities', 'type': 'services'},
    'TRA': {'name': 'Travel agency, tour operator, reservation service and related activities', 'type': 'services'},
    'SEC': {'name': 'Security and investigation activities', 'type': 'services'},
    'BUI': {'name': 'Services to buildings and landscape activities', 'type': 'services'},
    'OFS': {'name': 'Office administrative, office support and other business support activities', 'type': 'services'},
    'ADM': {'name': 'Public administration and defense; compulsory social security', 'type': 'services'},
    'EDU': {'name': 'Education', 'type': 'services'},
    'HEA': {'name': 'Human health activities', 'type': 'services'},
    'RES': {'name': 'Residential care activities', 'type': 'services'},
    'SOC': {'name': 'Social work activities without accommodation', 'type': 'services'},
    'CRE': {'name': 'Creative, arts and entertainment activities', 'type': 'services'},
    'LIB': {'name': 'Libraries, archives, museums and other cultural activities', 'type': 'services'},
    'BET': {'name': 'Gambling and betting activities', 'type': 'services'},
    'SPO': {'name': 'Sports activities and amusement and recreation activities', 'type': 'services'},
    'AMO': {'name': 'Activities of membership organizations', 'type': 'services'},
    'REP': {'name': 'Repair of computers and personal and household goods', 'type': 'services'},
    'OPS': {'name': 'Other personal service activities', 'type': 'services'},
    'IMP': {'name': 'Imports', 'type': 'imports'}
}

sector_groups = {
    'Agriculture': ['CRO', 'LIV', 'FOR', 'FIS'],
    'Mining and quarrying': ['MIN'],
    'Manufacturing': ['MFO', 'MBE', 'MTO', 'MTE', 'MWO', 'MPA', 'MCO', 'MCH', 'MPH', 'MRU', 'MNM', 'MBA', 'MFA', 'MEL', 'MMA', 'MTR', 'MFU', 'MAO'],
    'Construction': ['CON'],
    'Other sectors': ['ELE', 'WAS', 'TRW', 'TRR', 'TRL', 'POS', 'ACF', 'PUB', 'VID', 'TEL', 'INF', 'FIN', 'INS', 'REA', 'ACC', 'CST', 'ARC', 'VET', 'REN', 'EMP', 'TRA', 'SEC', 'BUI', 'OFS', 'ADM', 'EDU', 'HEA', 'RES', 'SOC', 'CRE', 'LIB', 'BET', 'SPO', 'AMO', 'REP', 'OPS']
}

sector_grouping = {}
for group, sectors in sector_groups.items():
    for sector in sectors:
        sector_grouping[sector] = group

regional_sector_data = []

for scenario in scenarios:
    folder_path = os.path.join(results_folder, scenario["folder"])
    sector_file = os.path.join(folder_path, "loss_per_region_sector_time.csv")  # Adjust filename as needed
    
    if os.path.exists(sector_file):
        df = pd.read_csv(sector_file)
        df["scenario"] = scenario["name"]
        df["folder"] = scenario["folder"]
        
        df['sector_group'] = df['sector'].map(sector_grouping)
        
        regional_sector_data.append(df)

# Concatenate all data
if regional_sector_data:
    rs_df = pd.concat(regional_sector_data, ignore_index=True)
    print(f"\nLoaded regional sector data: {len(rs_df)} records")
    print(f"Scenarios: {rs_df['scenario'].unique()}")
    print(f"Regions: {rs_df['region'].unique()}")
    print(f"Sectors: {len(rs_df['sector'].unique())} unique sectors")
    print(f"Time steps: {rs_df['time_step'].min()} to {rs_df['time_step'].max()}")
else:
    print("No regional sector data found. Please check file paths and names.")
    rs_df = pd.DataFrame()


Loaded regional sector data: 3120425 records
Scenarios: ['Simultaneous' 'Saigon only' 'Mekong only' 'Tonlè Sap only' 'Cascading'
 'Mixed']
Regions: [ 10201  10202  10203 ... 240202 240203 240204]
Sectors: 35 unique sectors
Time steps: 0 to 10


In [79]:
if not rs_df.empty:
    # Calculate total losses by scenario and simplified group (summing across regions and time)
    sector_group_totals = rs_df.groupby(['scenario', 'sector_group'])['loss'].sum().reset_index()
    
    # Use the scenario order from the original summary_df (if available) or maintain input order
    try:
        scenario_order = summary_df['scenario'].tolist()
    except:
        scenario_order = [s['name'] for s in scenarios]
    
    # Reorder the dataframe to match scenario order
    sector_group_totals['scenario'] = pd.Categorical(sector_group_totals['scenario'], categories=scenario_order, ordered=True)
    sector_group_totals = sector_group_totals.sort_values('scenario')
    
    fig5 = go.Figure()
    
    for group in ['Agriculture', 'Mining and quarrying', 'Manufacturing', 'Construction', 'Other sectors']:
        group_data = sector_group_totals[sector_group_totals['sector_group'] == group]
        if not group_data.empty:
            fig5.add_trace(go.Bar(
                name=group,
                x=group_data['scenario'],
                y=group_data['loss'],
                text=group_data['loss'].round(1),
                textposition='inside',
                textfont=dict(size=16)
            ))
    
    fig5.update_layout(
        xaxis_title='Scenario',
        yaxis_title='Household loss (million USD)',
        barmode='stack',
        height=600,
        width=1000,
        xaxis_tickangle=-45,
        showlegend=True,
        legend=dict(
            orientation="v",
            yanchor="top",
            y=1,
            xanchor="left",
            x=1.02,
        ),
        font=dict(size=18)
    )
    
    fig5.show()

In [80]:
if not rs_df.empty:
    sector_group_time_series = rs_df.groupby(['scenario', 'sector_group', 'time_step'])['loss'].sum().reset_index()
    
    fig6 = px.line(
        sector_group_time_series,
        x='time_step',
        y='loss',
        color='sector_group',
        facet_col='scenario',
        facet_col_wrap=2,
        height=900,
        width=1200,
        markers=True,
        labels={
            'time_step': 'Weeks', 
            'loss': 'Household loss (million USD)',
            'sector_group': 'Sectors'
        }
    )
    
    fig6.show()